# **Reducing RAM usage of Programs**

All our programs consume some amount of memory. But if this memory consumption is larger than our computer memory (i.e. RAM), the program will not be able to execute. Also in general more memory usage means higher level of overhead. Therefore we should always try to reduce the memory consumption of our programs as much as possible. We can do that by several ways like using better data structures, data compression techniques, avoiding unnecessary data movements etc.

For an example lets consider a list with same object in all indices vs a list with unique items in all indices and their memory cost.


In [1]:
%load_ext memory_profiler

In [2]:
%memit [0] * int(1e8)

peak memory: 849.66 MiB, increment: 763.10 MiB


In [2]:
%memit [n for n in range(int(1e8))]

peak memory: 4026.67 MiB, increment: 3940.16 MiB


We can see that list with unique items consume close to 4GB of memory. But if we use a different data stucture to store the same amount of data memory consumption is different.

In [3]:
import array
%memit array.array('l', range(int(1e8)))

peak memory: 712.28 MiB, increment: 624.44 MiB


As we can see it only consume ~600MB of memory. This result emphasize the importance of using proper data structure for right purpose. You may think that then usage of arrays would be great for anything. But it is not. It have limited set of supported data types. Also the moment we derefence our data from array, python will build a new object which will cost memory. So if your program is going to deference data frequently and process no memory saving will occur.

If we need array with more data type support, we can use numpy arrays. They act very simialar to the normal python arrays and provides better data type support.



In [7]:
import numpy as np
%memit arr=np.ones(int(1e8), np.int8)

peak memory: 289.00 MiB, increment: 95.37 MiB
